In [ ]:
import pandas as pd
import time
import tracemalloc
import random
from typing import List, Dict, Any

In [ ]:
def bubble_sort(arr: List[Dict], key: str = None) -> List[Dict]:
    """تنفيذ خوارزمية Bubble Sort"""
    n = len(arr)
    data = arr.copy()

    for i in range(n):
        swapped = False
        for j in range(0, n - i - 1):
            if key:
                val_j = data[j][key]
                val_j1 = data[j + 1][key]
            else:
                val_j = data[j]
                val_j1 = data[j + 1]
            
            if val_j > val_j1:
                data[j], data[j + 1] = data[j + 1], data[j]
                swapped = True
        if not swapped:
            break
    
    return data

In [ ]:
def quick_sort(arr: List[Dict], key: str = None) -> List[Dict]:
    """تنفيذ خوارزمية Quick Sort"""
    if len(arr) <= 1:
        return arr.copy()
    
    pivot_idx = random.randint(0, len(arr) - 1)
    pivot = arr[pivot_idx]
    pivot_val = pivot[key] if key else pivot
    
    left, middle, right = [], [], []
    
    for item in arr:
        item_val = item[key] if key else item
        if item_val < pivot_val:
            left.append(item)
        elif item_val == pivot_val:
            middle.append(item)
        else:
            right.append(item)
    
    return quick_sort(left, key) + middle + quick_sort(right, key)


In [ ]:
def measure_performance(algorithm, data: List[Dict], key: str = None) -> Dict[str, Any]:
    """قياس أداء الخوارزمية"""
    tracemalloc.start()
    start_time = time.perf_counter()
    
    sorted_data = algorithm(data, key)
    
    end_time = time.perf_counter()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    return {
        'sorted_data': sorted_data,
        'execution_time': end_time - start_time,
        'memory_peak_kb': peak / 1024,
        'data_size': len(data),
        'algorithm': algorithm.__name__
    }

In [ ]:
def compare_algorithms(data: List[Dict], key: str = None) -> Dict[str, Any]:
    """مقارنة أداء الخوارزميات"""
    results = {}
    
    # قياس أداء Bubble Sort
    bubble_result = measure_performance(bubble_sort, data, key)
    results['bubble_sort'] = bubble_result
    
    # قياس أداء Quick Sort
    quick_result = measure_performance(quick_sort, data, key)
    results['quick_sort'] = quick_result
    
    # تحديد الفائز
    if bubble_result['execution_time'] < quick_result['execution_time']:
        results['winner'] = 'Bubble Sort'
        results['improvement_ratio'] = quick_result['execution_time'] / bubble_result['execution_time']
    else:
        results['winner'] = 'Quick Sort' 
        results['improvement_ratio'] = bubble_result['execution_time'] / quick_result['execution_time']
    
    return results

In [ ]:
def load_and_prepare_data(csv_file_path: str):
    """تحميل بياناتك الحقيقية من CSV"""
    
    print(f"📁 جاري تحميل البيانات من: {csv_file_path}")
    
    # تحميل البيانات
    df = pd.read_csv(csv_file_path)
    
    # عرض معلومات عن البيانات
    print(f"✅ تم تحميل {len(df)} سجل بنجاح")
    print("\n📊 معلومات الأعمدة:")
    for i, col in enumerate(df.columns, 1):
        print(f"   {i}. {col} - {df[col].dtype} - {df[col].notna().sum()} قيمة")
    
    # تنظيف البيانات الأساسية
    print("\n🔧 تنظيف البيانات...")
    
    # التأكد من أن العمود Price رقمي
    if 'Price' in df.columns:
        df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
        df = df.dropna(subset=['Price'])
    
    # تحويل البيانات إلى قواميس للخوارزميات
    data_dict = df.to_dict('records')
    
    return df, data_dict

In [ ]:
def create_real_data_scenarios(data_dict: List[Dict], df: pd.DataFrame):
    """إنشاء سيناريوهات اختبار من بياناتك الحقيقية"""
    
    scenarios = {}
    
    # 1. البيانات الكاملة
    scenarios['البيانات_الكاملة'] = data_dict
    
    # 2. بيانات حسب السعر (مرتبة)
    scenarios['شبه_مرتب_حسب_السعر'] = sorted(data_dict, key=lambda x: x.get('Price', 0))
    
    # 3. بيانات عشوائية
    scenarios['عشوائي'] = data_dict.copy()
    random.shuffle(scenarios['عشوائي'])
    
    # 4. بيانات مرتبة عكسياً
    scenarios['عكسي_حسب_السعر'] = sorted(data_dict, key=lambda x: x.get('Price', 0), reverse=True)
    
    # 5. مجموعات بحجوم مختلفة
    if len(data_dict) >= 1000:
        scenarios['بيانات_صغيرة'] = data_dict[:100]
        scenarios['بيانات_متوسطة'] = data_dict[:500]
        scenarios['بيانات_كبيرة'] = data_dict[:1000]
    
    # 6. بيانات حسب أنواع التذاكر
    if 'Ticket Type' in df.columns:
        for ticket_type in df['Ticket Type'].unique()[:3]:  # أول 3 أنواع فقط
            type_data = [item for item in data_dict if item.get('Ticket Type') == ticket_type]
            if len(type_data) > 10:  # إذا كان هناك بيانات كافية
                scenarios[f'نوع_تذكرة_{ticket_type}'] = type_data
    
    # 7. بيانات حسب المحطات
    if 'Departure Station' in df.columns:
        top_stations = df['Departure Station'].value_counts().head(2).index
        for station in top_stations:
            station_data = [item for item in data_dict if item.get('Departure Station') == station]
            if len(station_data) > 10:
                scenarios[f'محطة_{station}'] = station_data
    
    return scenarios

In [ ]:
def run_comprehensive_analysis(csv_file_path: str):
    """تشغيل تحليل شامل على بياناتك الحقيقية"""
    
    print("🚀 بدء التحليل الشامل على بياناتك الحقيقية")
    print("=" * 60)
    
    # تحميل البيانات
    df, data_dict = load_and_prepare_data(csv_file_path)
    
    if not data_dict:
        print("❌ لا توجد بيانات لتحليلها")
        return
    
    # إنشاء سيناريوهات الاختبار
    scenarios = create_real_data_scenarios(data_dict, df)
    
    all_results = {}
    
    # اختيار الأعمدة المناسبة للترتيب
    sort_columns = []
    if 'Price' in df.columns:
        sort_columns.append(('Price', 'السعر'))
    if 'Date of Journey' in df.columns:
        sort_columns.append(('Date of Journey', 'تاريخ الرحلة'))
    if 'Date of Purchase' in df.columns:
        sort_columns.append(('Date of Purchase', 'تاريخ الشراء'))
    
    # إذا لم توجد أعمدة مناسبة، استخدم أول عمود رقمي
    if not sort_columns:
        numeric_cols = df.select_dtypes(include=['number']).columns
        if len(numeric_cols) > 0:
            sort_columns.append((numeric_cols[0], numeric_cols[0]))
    
    for sort_col, col_name in sort_columns:
        print(f"\n🎯 التحليل حسب: {col_name}")
        print("-" * 40)
        
        for scenario_name, scenario_data in scenarios.items():
            if len(scenario_data) < 2:
                continue
                
            print(f"\n📊 السيناريو: {scenario_name}")
            print(f"   حجم البيانات: {len(scenario_data)} سجل")
            
            try:
                # المقارنة بين الخوارزميات
                results = compare_algorithms(scenario_data, sort_col)
                all_results[f"{scenario_name}_{sort_col}"] = results
                
                # عرض النتائج
                bubble_time = results['bubble_sort']['execution_time']
                quick_time = results['quick_sort']['execution_time']
                
                print(f"   ⏱️  وقت التنفيذ:")
                print(f"     - Bubble Sort: {bubble_time:.6f} ثانية")
                print(f"     - Quick Sort: {quick_time:.6f} ثانية")
                print(f"   🏆 الفائز: {results['winner']}")
                print(f"   📈 نسبة التحسن: {results['improvement_ratio']:.2f}x")
                
            except Exception as e:
                print(f"   ❌ خطأ في المعالجة: {e}")
                continue
    
    return all_results, df

In [ ]:

def display_final_report(results: Dict, df: pd.DataFrame):
    """عرض التقرير النهائي"""
    
    print("\n" + "=" * 70)
    print("📈 التقرير النهائي - تحليل أداء الخوارزميات على بياناتك الحقيقية")
    print("=" * 70)
    
    # إحصائيات البيانات
    print(f"\n📊 إحصائيات البيانات:")
    print(f"   إجمالي السجلات: {len(df):,}")
    print(f"   عدد الأعمدة: {len(df.columns)}")
    
    if 'Price' in df.columns:
        prices = df['Price'].dropna()
        print(f"   مدى الأسعار: {prices.min():.2f} - {prices.max():.2f}")
        print(f"   متوسط السعر: {prices.mean():.2f}")
    
    # تحليل النتائج
    print(f"\n🎯 تحليل النتائج:")
    bubble_wins = 0
    quick_wins = 0
    
    for scenario, result in results.items():
        if result['winner'] == 'Bubble Sort':
            bubble_wins += 1
        else:
            quick_wins += 1
    
    print(f"   عدد مرات فوز Bubble Sort: {bubble_wins}")
    print(f"   عدد مرات فوز Quick Sort: {quick_wins}")
    print(f"   الخوارزمية الأفضل إجمالاً: {'Bubble Sort' if bubble_wins >= quick_wins else 'Quick Sort'}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

def create_performance_charts(results, df, output_dir="charts"):
    """Create comprehensive performance comparison charts"""
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Set style for better looking charts
    plt.style.use('seaborn-v0_8')
    sns.set_palette("husl")
    
    # 1. Execution Time Comparison Chart
    plt.figure(figsize=(15, 10))
    
    # Prepare data for plotting
    scenarios = []
    bubble_times = []
    quick_times = []
    improvements = []
    
    for scenario_name, result in results.items():
        scenarios.append(scenario_name[:25] + "..." if len(scenario_name) > 25 else scenario_name)
        bubble_times.append(result['bubble_sort']['execution_time'])
        quick_times.append(result['quick_sort']['execution_time'])
        improvements.append(result['improvement_ratio'])
    
    x = np.arange(len(scenarios))
    width = 0.35
    
    # Chart 1: Execution Time Comparison
    plt.subplot(2, 2, 1)
    bars1 = plt.bar(x - width/2, bubble_times, width, label='Bubble Sort', color='#FF6B6B', alpha=0.8)
    bars2 = plt.bar(x + width/2, quick_times, width, label='Quick Sort', color='#4ECDC4', alpha=0.8)
    
    plt.xlabel('Scenarios')
    plt.ylabel('Execution Time (seconds)')
    plt.title('Execution Time: Bubble Sort vs Quick Sort', fontsize=12, fontweight='bold')
    plt.xticks(x, scenarios, rotation=45, ha='right')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Add values on bars
    for bar in bars1:
        height = bar.get_height()
        if height > 0.001:  # Only label if significant
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.4f}s', ha='center', va='bottom', fontsize=8, fontweight='bold')
    
    for bar in bars2:
        height = bar.get_height()
        if height > 0.001:
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.4f}s', ha='center', va='bottom', fontsize=8, fontweight='bold')
    
    # Chart 2: Performance Improvement Ratio
    plt.subplot(2, 2, 2)
    colors = ['#2E8B57' if x > 1 else '#DC143C' for x in improvements]
    bars = plt.bar(scenarios, improvements, color=colors, alpha=0.8)
    
    plt.axhline(y=1, color='black', linestyle='--', alpha=0.7, linewidth=1)
    plt.xlabel('Scenarios')
    plt.ylabel('Improvement Ratio (x times)')
    plt.title('Quick Sort Performance Improvement Over Bubble Sort', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    
    # Add values on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}x', ha='center', va='bottom', fontweight='bold')
    
    # Chart 3: Memory Usage Comparison
    plt.subplot(2, 2, 3)
    
    bubble_memory = [result['bubble_sort']['memory_peak_kb'] for result in results.values()]
    quick_memory = [result['quick_sort']['memory_peak_kb'] for result in results.values()]
    
    bars1 = plt.bar(x - width/2, bubble_memory, width, label='Bubble Sort', color='#FFA07A', alpha=0.8)
    bars2 = plt.bar(x + width/2, quick_memory, width, label='Quick Sort', color='#20B2AA', alpha=0.8)
    
    plt.xlabel('Scenarios')
    plt.ylabel('Memory Usage (KB)')
    plt.title('Memory Usage Comparison', fontsize=12, fontweight='bold')
    plt.xticks(x, scenarios, rotation=45, ha='right')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Add values on bars
    for bar in bars1:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.0f}KB', ha='center', va='bottom', fontsize=8, fontweight='bold')
    
    for bar in bars2:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.0f}KB', ha='center', va='bottom', fontsize=8, fontweight='bold')
    
    # Chart 4: Winners Distribution
    plt.subplot(2, 2, 4)
    
    winners = [result['winner'] for result in results.values()]
    winner_counts = pd.Series(winners).value_counts()
    
    colors = ['#FF6B6B' if winner == 'Bubble Sort' else '#4ECDC4' for winner in winner_counts.index]
    plt.pie(winner_counts.values, labels=winner_counts.index, autopct='%1.1f%%', 
            colors=colors, startangle=90, textprops={'fontweight': 'bold'})
    plt.title('Winning Algorithm Distribution', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/performance_comparison.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

def create_scalability_chart(results, output_dir="charts"):
    """Create scalability analysis chart"""
    
    plt.figure(figsize=(12, 8))
    
    # Extract size and performance data
    sizes = []
    bubble_times = []
    quick_times = []
    scenario_names = []
    
    for scenario_name, result in results.items():
        size = result['bubble_sort']['data_size']
        sizes.append(size)
        bubble_times.append(result['bubble_sort']['execution_time'])
        quick_times.append(result['quick_sort']['execution_time'])
        scenario_names.append(scenario_name)
    
    # Sort data by size
    sorted_data = sorted(zip(sizes, bubble_times, quick_times, scenario_names))
    sizes, bubble_times, quick_times, scenario_names = zip(*sorted_data)
    
    plt.plot(sizes, bubble_times, 'o-', label='Bubble Sort', linewidth=3, markersize=10, 
             color='#FF6B6B', markerfacecolor='white', markeredgewidth=2)
    plt.plot(sizes, quick_times, 's-', label='Quick Sort', linewidth=3, markersize=10,
             color='#4ECDC4', markerfacecolor='white', markeredgewidth=2)
    
    plt.xlabel('Dataset Size (number of records)', fontsize=12, fontweight='bold')
    plt.ylabel('Execution Time (seconds)', fontsize=12, fontweight='bold')
    plt.title('Scalability Analysis: Algorithm Performance vs Dataset Size', 
              fontsize=14, fontweight='bold')
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    
    # Add data point labels for significant points
    for i, (size, b_time, q_time, name) in enumerate(zip(sizes, bubble_times, quick_times, scenario_names)):
        if i % max(1, len(sizes)//5) == 0:  # Label every ~5th point
            plt.annotate(f'{size} rec', (size, b_time), textcoords="offset points", 
                        xytext=(0,10), ha='center', fontsize=9, fontweight='bold',
                        bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/scalability_analysis.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

def create_algorithm_behavior_chart(results, output_dir="charts"):
    """Create algorithm behavior analysis by data type"""
    
    plt.figure(figsize=(14, 8))
    
    # Categorize scenarios
    scenario_categories = {
        'Random Data': [],
        'Nearly Sorted': [],
        'Reverse Sorted': [],
        'Small Dataset': [],
        'Large Dataset': []
    }
    
    for scenario_name, result in results.items():
        scenario_lower = scenario_name.lower()
        
        if 'random' in scenario_lower:
            scenario_categories['Random Data'].append(result)
        elif 'nearly' in scenario_lower or 'sorted' in scenario_lower:
            scenario_categories['Nearly Sorted'].append(result)
        elif 'reverse' in scenario_lower:
            scenario_categories['Reverse Sorted'].append(result)
        elif 'small' in scenario_lower:
            scenario_categories['Small Dataset'].append(result)
        elif 'large' in scenario_lower or 'big' in scenario_lower:
            scenario_categories['Large Dataset'].append(result)
    
    # Calculate average performance per category
    categories = []
    bubble_avg_times = []
    quick_avg_times = []
    bubble_avg_memory = []
    quick_avg_memory = []
    
    for category, category_results in scenario_categories.items():
        if category_results:
            categories.append(category)
            bubble_avg_times.append(np.mean([r['bubble_sort']['execution_time'] for r in category_results]))
            quick_avg_times.append(np.mean([r['quick_sort']['execution_time'] for r in category_results]))
            bubble_avg_memory.append(np.mean([r['bubble_sort']['memory_peak_kb'] for r in category_results]))
            quick_avg_memory.append(np.mean([r['quick_sort']['memory_peak_kb'] for r in category_results]))
    
    x = np.arange(len(categories))
    width = 0.35
    
    # Chart 1: Time comparison by category
    plt.subplot(1, 2, 1)
    bars1 = plt.bar(x - width/2, bubble_avg_times, width, label='Bubble Sort', 
                   color='#FF6B6B', edgecolor='black', linewidth=1)
    bars2 = plt.bar(x + width/2, quick_avg_times, width, label='Quick Sort', 
                   color='#4ECDC4', edgecolor='black', linewidth=1)
    
    plt.xlabel('Data Categories', fontweight='bold')
    plt.ylabel('Average Execution Time (seconds)', fontweight='bold')
    plt.title('Average Performance by Data Category', fontsize=12, fontweight='bold')
    plt.xticks(x, categories, rotation=15)
    plt.legend()
    plt.grid(True, alpha=0.3, axis='y')
    
    # Add values on bars
    for bar in bars1:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}s', ha='center', va='bottom', fontweight='bold')
    
    for bar in bars2:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}s', ha='center', va='bottom', fontweight='bold')
    
    # Chart 2: Memory comparison by category
    plt.subplot(1, 2, 2)
    bars1 = plt.bar(x - width/2, bubble_avg_memory, width, label='Bubble Sort', 
                   color='#FFA07A', edgecolor='black', linewidth=1)
    bars2 = plt.bar(x + width/2, quick_avg_memory, width, label='Quick Sort', 
                   color='#20B2AA', edgecolor='black', linewidth=1)
    
    plt.xlabel('Data Categories', fontweight='bold')
    plt.ylabel('Average Memory Usage (KB)', fontweight='bold')
    plt.title('Average Memory Usage by Data Category', fontsize=12, fontweight='bold')
    plt.xticks(x, categories, rotation=15)
    plt.legend()
    plt.grid(True, alpha=0.3, axis='y')
    
    # Add values on bars
    for bar in bars1:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.0f}KB', ha='center', va='bottom', fontweight='bold')
    
    for bar in bars2:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.0f}KB', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/algorithm_behavior.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

def create_memory_time_tradeoff(results, output_dir="charts"):
    """Create memory-time tradeoff analysis"""
    
    plt.figure(figsize=(12, 8))
    
    bubble_times = [result['bubble_sort']['execution_time'] for result in results.values()]
    bubble_memory = [result['bubble_sort']['memory_peak_kb'] for result in results.values()]
    quick_times = [result['quick_sort']['execution_time'] for result in results.values()]
    quick_memory = [result['quick_sort']['memory_peak_kb'] for result in results.values()]
    scenario_names = list(results.keys())
    
    # Create scatter plot
    bubble_scatter = plt.scatter(bubble_times, bubble_memory, color='red', s=150, 
                               alpha=0.8, label='Bubble Sort', edgecolors='black', linewidth=1)
    quick_scatter = plt.scatter(quick_times, quick_memory, color='#4ECDC4', s=150, 
                              alpha=0.8, label='Quick Sort', edgecolors='black', linewidth=1)
    
    # Add trend lines
    if len(bubble_times) > 1:
        z_bubble = np.polyfit(bubble_times, bubble_memory, 1)
        p_bubble = np.poly1d(z_bubble)
        plt.plot(bubble_times, p_bubble(bubble_times), "--", color='#FF6B6B', alpha=0.5)
    
    if len(quick_times) > 1:
        z_quick = np.polyfit(quick_times, quick_memory, 1)
        p_quick = np.poly1d(z_quick)
        plt.plot(quick_times, p_quick(quick_times), "--", color='#4ECDC4', alpha=0.5)
    
    plt.xlabel('Execution Time (seconds)', fontweight='bold')
    plt.ylabel('Memory Usage (KB)', fontweight='bold')
    plt.title('Memory-Time Tradeoff Analysis', fontsize=14, fontweight='bold')
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    
    # Add scenario numbers for key points
    for i, (scenario_name, b_time, b_mem, q_time, q_mem) in enumerate(zip(scenario_names, bubble_times, bubble_memory, quick_times, quick_memory)):
        if i % max(1, len(scenario_names)//4) == 0:  # Label key points
            plt.annotate(f'S{i+1}', (b_time, b_mem), textcoords="offset points", 
                        xytext=(5,5), ha='left', fontweight='bold',
                        bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
            plt.annotate(f'S{i+1}', (q_time, q_mem), textcoords="offset points", 
                        xytext=(5,5), ha='left', fontweight='bold',
                        bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/memory_time_tradeoff.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

def generate_comprehensive_report(results, df, output_dir="charts"):
    """Generate comprehensive visual report"""
    
    print("\n📊 Generating comprehensive visual reports...")
    
    # Create all charts
    create_performance_charts(results, df, output_dir)
    create_scalability_chart(results, output_dir)
    create_algorithm_behavior_chart(results, output_dir)
    create_memory_time_tradeoff(results, output_dir)
    
    print(f"✅ Visual reports generated in: {output_dir}/")

def main():
    """Main execution function"""
    
    # Your CSV file path
    csv_file_path = "cleaned_data.csv"
    
    try:
        # Comprehensive analysis
        results, df = run_comprehensive_analysis(csv_file_path)
        
        if results:
            # Display final report
            display_final_report(results, df)
            
            # Generate visual reports
            generate_comprehensive_report(results, df)
            
            # Save text results
            output_file = "algorithm_analysis_results.txt"
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write("ALGORITHM PERFORMANCE ANALYSIS RESULTS\n")
                f.write("=" * 50 + "\n")
                f.write(f"Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Total Records: {len(df):,}\n")
                f.write(f"Number of Scenarios: {len(results)}\n\n")
                
                for scenario, result in results.items():
                    f.write(f"SCENARIO: {scenario}\n")
                    f.write(f"  Winner: {result['winner']}\n")
                    f.write(f"  Bubble Sort Time: {result['bubble_sort']['execution_time']:.6f} seconds\n")
                    f.write(f"  Quick Sort Time: {result['quick_sort']['execution_time']:.6f} seconds\n")
                    f.write(f"  Bubble Sort Memory: {result['bubble_sort']['memory_peak_kb']:.2f} KB\n")
                    f.write(f"  Quick Sort Memory: {result['quick_sort']['memory_peak_kb']:.2f} KB\n")
                    f.write(f"  Improvement Ratio: {result['improvement_ratio']:.2f}x\n\n")
                
                # Overall statistics
                bubble_wins = sum(1 for r in results.values() if r['winner'] == 'Bubble Sort')
                quick_wins = sum(1 for r in results.values() if r['winner'] == 'Quick Sort')
                
                f.write("\nFINAL STATISTICS:\n")
                f.write(f"  Bubble Sort Wins: {bubble_wins}\n")
                f.write(f"  Quick Sort Wins: {quick_wins}\n")
                f.write(f"  Best Overall Algorithm: {'Bubble Sort' if bubble_wins > quick_wins else 'Quick Sort'}\n")
            
            print(f"\n💾 Text results saved to: {output_file}")
            print("📈 Visual reports generated in 'charts/' folder")
        
        print("\n" + "=" * 70)
        print("✅ Analysis completed successfully!")
        print("=" * 70)
        
    except FileNotFoundError:
        print(f"❌ File {csv_file_path} not found")
        print("⚠️ Please update the file path in main() function")
    except Exception as e:
        print(f"❌ Error occurred: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

In [ ]:
def analyze_bubble_sort_complexity():
    """Analyze Bubble Sort complexity line by line"""
    
    complexity_analysis = {
        'algorithm': 'Bubble Sort',
        'line_by_line': [
            {
                'line': 'for i in range(n):',
                'complexity': 'O(n)',
                'explanation': 'Outer loop runs n times'
            },
            {
                'line': 'for j in range(0, n - i - 1):',
                'complexity': 'O(n)',
                'explanation': 'Inner loop runs n-i-1 times, average O(n)'
            },
            {
                'line': 'if arr[j] > arr[j + 1]:',
                'complexity': 'O(1)',
                'explanation': 'Comparison operation - constant time'
            },
            {
                'line': 'arr[j], arr[j + 1] = arr[j + 1], arr[j]',
                'complexity': 'O(1)',
                'explanation': 'Swap operation - constant time'
            }
        ],
        'total_time_complexity': {
            'best_case': 'O(n)',
            'average_case': 'O(n²)',
            'worst_case': 'O(n²)',
            'explanation': 'Nested loops: O(n) * O(n) = O(n²)'
        },
        'space_complexity': {
            'complexity': 'O(1)',
            'explanation': 'Only constant extra space for variables'
        }
    }
    
    return complexity_analysis

def analyze_quick_sort_complexity():
    """Analyze Quick Sort complexity line by line"""
    
    complexity_analysis = {
        'algorithm': 'Quick Sort',
        'line_by_line': [
            {
                'line': 'if len(arr) <= 1: return arr',
                'complexity': 'O(1)',
                'explanation': 'Base case - constant time'
            },
            {
                'line': 'pivot = arr[len(arr)//2]',
                'complexity': 'O(1)',
                'explanation': 'Pivot selection - constant time'
            },
            {
                'line': 'for item in arr:',
                'complexity': 'O(n)',
                'explanation': 'Partitioning loop runs n times'
            },
            {
                'line': 'quick_sort(left) + middle + quick_sort(right)',
                'complexity': 'O(log n) * O(n)',
                'explanation': 'Recursive calls with partitioning'
            }
        ],
        'total_time_complexity': {
            'best_case': 'O(n log n)',
            'average_case': 'O(n log n)',
            'worst_case': 'O(n²)',
            'explanation': 'Average: O(n) partitioning * O(log n) levels = O(n log n)'
        },
        'space_complexity': {
            'complexity': 'O(log n)',
            'explanation': 'Recursive call stack depth in best case'
        }
    }
    
    return complexity_analysis

In [ ]:
def generate_complexity_report():
    """Generate comprehensive complexity analysis report"""
    
    print("🔍 COMPUTATIONAL COMPLEXITY ANALYSIS")
    print("=" * 60)
    
    # Analyze both algorithms
    bubble_analysis = analyze_bubble_sort_complexity()
    quick_analysis = analyze_quick_sort_complexity()
    
    # Print Bubble Sort analysis
    print(f"\n📊 {bubble_analysis['algorithm']} - Line by Line Analysis:")
    print("-" * 50)
    for line_analysis in bubble_analysis['line_by_line']:
        print(f"   {line_analysis['line']:40} → {line_analysis['complexity']:8} | {line_analysis['explanation']}")
    
    print(f"\n   Time Complexity Summary:")
    print(f"     Best Case:    {bubble_analysis['total_time_complexity']['best_case']}")
    print(f"     Average Case: {bubble_analysis['total_time_complexity']['average_case']}")
    print(f"     Worst Case:   {bubble_analysis['total_time_complexity']['worst_case']}")
    print(f"     Space:        {bubble_analysis['space_complexity']['complexity']}")
    
    # Print Quick Sort analysis
    print(f"\n📊 {quick_analysis['algorithm']} - Line by Line Analysis:")
    print("-" * 50)
    for line_analysis in quick_analysis['line_by_line']:
        print(f"   {line_analysis['line']:40} → {line_analysis['complexity']:8} | {line_analysis['explanation']}")
    
    print(f"\n   Time Complexity Summary:")
    print(f"     Best Case:    {quick_analysis['total_time_complexity']['best_case']}")
    print(f"     Average Case: {quick_analysis['total_time_complexity']['average_case']}")
    print(f"     Worst Case:   {quick_analysis['total_time_complexity']['worst_case']}")
    print(f"     Space:        {quick_analysis['space_complexity']['complexity']}")
    
    return bubble_analysis, quick_analysis

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

def theoretical_time_bubble(n):
    """Theoretical time complexity for Bubble Sort: O(n²)"""
    return 0.000001 * n ** 2  # Adjusted coefficient for realistic values

def theoretical_time_quick(n):
    """Theoretical time complexity for Quick Sort: O(n log n)"""
    return 0.000001 * n * np.log2(n + 1)  # Avoid log(0)

def analyze_growth_rates():
    """Analyze order of growth with mathematical models"""
    
    print("\n📈 ORDER OF GROWTH ANALYSIS")
    print("=" * 60)
    
    # Sample sizes for analysis
    sizes = np.array([100, 500, 1000, 2000, 5000, 10000])
    
    print(f"\n📐 Theoretical Time Complexity Models:")
    print(f"{'Size (n)':>10} {'Bubble O(n²)':>15} {'Quick O(n log n)':>18} {'Ratio':>10}")
    print("-" * 60)
    
    for n in sizes:
        t_bubble = theoretical_time_bubble(n)
        t_quick = theoretical_time_quick(n)
        ratio = t_bubble / t_quick if t_quick > 0 else float('inf')
        
        print(f"{n:10} {t_bubble:15.6f} {t_quick:18.6f} {ratio:10.2f}x")
    
    # Growth rate analysis
    print(f"\n📊 Growth Rate Analysis:")
    for i in range(1, len(sizes)):
        n1, n2 = sizes[i-1], sizes[i]
        bubble_growth = theoretical_time_bubble(n2) / theoretical_time_bubble(n1)
        quick_growth = theoretical_time_quick(n2) / theoretical_time_quick(n1)
        
        print(f"  n={n1}→{n2}: Bubble growth: {bubble_growth:.2f}x, Quick growth: {quick_growth:.2f}x")
    
    return sizes, theoretical_time_bubble(sizes), theoretical_time_quick(sizes)

In [ ]:
def measure_algorithm_performance(algorithm, data_sizes, num_trials=5):
    """Measure actual performance with multiple trials"""
    
    results = {
        'times': [],
        'memory': [],
        'sizes': data_sizes
    }
    
    for size in data_sizes:
        trial_times = []
        trial_memory = []
        
        for trial in range(num_trials):
            # Generate test data
            test_data = [random.randint(1, 10000) for _ in range(size)]
            
            # Measure time and memory
            tracemalloc.start()
            start_time = time.perf_counter()
            
            algorithm(test_data.copy())
            
            end_time = time.perf_counter()
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            
            execution_time = (end_time - start_time) * 1000  # Convert to milliseconds
            memory_used = peak / 1024  # Convert to KB
            
            trial_times.append(execution_time)
            trial_memory.append(memory_used)
        
        # Store average results
        results['times'].append(np.mean(trial_times))
        results['memory'].append(np.mean(trial_memory))
    
    return results

def run_experimental_analysis():
    """Run comprehensive experimental analysis"""
    
    print("\n⏱️ EXPERIMENTAL PERFORMANCE ANALYSIS")
    print("=" * 60)
    
    # Test sizes - adjust based on your system capabilities
    data_sizes = [100, 500, 1000, 2000, 5000]
    
    print(f"\n🔬 Testing with sizes: {data_sizes}")
    print(f"   Number of trials per size: 5")
    print(f"   Time unit: milliseconds")
    print(f"   Memory unit: KB")
    
    # Measure Bubble Sort performance
    print(f"\n📊 Bubble Sort Experimental Results:")
    print(f"{'Size':>8} {'Time (ms)':>12} {'Memory (KB)':>12} {'Theoretical':>12}")
    print("-" * 50)
    
    bubble_results = measure_algorithm_performance(bubble_sort, data_sizes)
    for i, size in enumerate(data_sizes):
        theoretical = theoretical_time_bubble(size) * 1000  # Convert to ms
        print(f"{size:8} {bubble_results['times'][i]:12.3f} {bubble_results['memory'][i]:12.1f} {theoretical:12.3f}")
    
    # Measure Quick Sort performance
    print(f"\n📊 Quick Sort Experimental Results:")
    print(f"{'Size':>8} {'Time (ms)':>12} {'Memory (KB)':>12} {'Theoretical':>12}")
    print("-" * 50)
    
    quick_results = measure_algorithm_performance(quick_sort, data_sizes)
    for i, size in enumerate(data_sizes):
        theoretical = theoretical_time_quick(size) * 1000  # Convert to ms
        print(f"{size:8} {quick_results['times'][i]:12.3f} {quick_results['memory'][i]:12.1f} {theoretical:12.3f}")
    
    return bubble_results, quick_results, data_sizes

In [ ]:
def create_growth_comparison_charts(bubble_results, quick_results, data_sizes):
    """Create diagrams showing running times and order of growth"""
    
    plt.figure(figsize=(15, 10))
    
    # Chart 1: Actual Running Time Comparison
    plt.subplot(2, 2, 1)
    plt.plot(data_sizes, bubble_results['times'], 'ro-', linewidth=2, markersize=8, label='Bubble Sort (Actual)')
    plt.plot(data_sizes, quick_results['times'], 'bo-', linewidth=2, markersize=8, label='Quick Sort (Actual)')
    
    # Add theoretical curves
    theoretical_sizes = np.linspace(min(data_sizes), max(data_sizes), 100)
    plt.plot(theoretical_sizes, [theoretical_time_bubble(n) * 1000 for n in theoretical_sizes], 
             'r--', alpha=0.7, label='Bubble Sort (Theoretical O(n²))')
    plt.plot(theoretical_sizes, [theoretical_time_quick(n) * 1000 for n in theoretical_sizes], 
             'b--', alpha=0.7, label='Quick Sort (Theoretical O(n log n))')
    
    plt.xlabel('Dataset Size (n)')
    plt.ylabel('Execution Time (milliseconds)')
    plt.title('Actual vs Theoretical Running Time Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')  # Log scale for better visualization
    
    # Chart 2: Order of Growth Comparison
    plt.subplot(2, 2, 2)
    
    # Calculate growth rates
    bubble_growth = [bubble_results['times'][i]/bubble_results['times'][0] for i in range(len(data_sizes))]
    quick_growth = [quick_results['times'][i]/quick_results['times'][0] for i in range(len(data_sizes))]
    theoretical_bubble_growth = [theoretical_time_bubble(n)/theoretical_time_bubble(data_sizes[0]) for n in data_sizes]
    theoretical_quick_growth = [theoretical_time_quick(n)/theoretical_time_quick(data_sizes[0]) for n in data_sizes]
    
    plt.plot(data_sizes, bubble_growth, 'ro-', linewidth=2, label='Bubble Sort Growth')
    plt.plot(data_sizes, quick_growth, 'bo-', linewidth=2, label='Quick Sort Growth')
    plt.plot(data_sizes, theoretical_bubble_growth, 'r--', alpha=0.7, label='Theoretical O(n²)')
    plt.plot(data_sizes, theoretical_quick_growth, 'b--', alpha=0.7, label='Theoretical O(n log n)')
    
    plt.xlabel('Dataset Size (n)')
    plt.ylabel('Relative Growth (times)')
    plt.title('Order of Growth Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Chart 3: Memory Usage Comparison
    plt.subplot(2, 2, 3)
    plt.plot(data_sizes, bubble_results['memory'], 'go-', linewidth=2, markersize=6, label='Bubble Sort Memory')
    plt.plot(data_sizes, quick_results['memory'], 'mo-', linewidth=2, markersize=6, label='Quick Sort Memory')
    
    plt.xlabel('Dataset Size (n)')
    plt.ylabel('Memory Usage (KB)')
    plt.title('Space Complexity Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Chart 4: Performance Ratio
    plt.subplot(2, 2, 4)
    performance_ratio = [bubble_results['times'][i] / quick_results['times'][i] for i in range(len(data_sizes))]
    plt.plot(data_sizes, performance_ratio, 'ko-', linewidth=2, markersize=6)
    
    plt.xlabel('Dataset Size (n)')
    plt.ylabel('Bubble Time / Quick Time')
    plt.title('Performance Ratio: How much faster is Quick Sort?')
    plt.grid(True, alpha=0.3)
    
    # Add ratio values on points
    for i, (size, ratio) in enumerate(zip(data_sizes, performance_ratio)):
        plt.annotate(f'{ratio:.1f}x', (size, ratio), textcoords="offset points", 
                    xytext=(0,10), ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('growth_analysis_comprehensive.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def compare_with_builtin_sort(data_sizes, num_trials=5):
    """Compare with Python's built-in sorted() function"""
    
    print(f"\n🏆 COMPARISON WITH BUILT-IN SORT (TimSort)")
    print("=" * 50)
    
    builtin_results = {'times': [], 'memory': []}
    
    for size in data_sizes:
        trial_times = []
        trial_memory = []
        
        for trial in range(num_trials):
            test_data = [random.randint(1, 10000) for _ in range(size)]
            
            tracemalloc.start()
            start_time = time.perf_counter()
            
            sorted(test_data)  # Python's built-in TimSort
            
            end_time = time.perf_counter()
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            
            trial_times.append((end_time - start_time) * 1000)
            trial_memory.append(peak / 1024)
        
        builtin_results['times'].append(np.mean(trial_times))
        builtin_results['memory'].append(np.mean(trial_memory))
    
    # Print comparison table
    print(f"\n{'Size':>8} {'Bubble (ms)':>12} {'Quick (ms)':>12} {'Built-in (ms)':>14} {'Quick/Built-in':>15}")
    print("-" * 70)
    
    bubble_results, quick_results, _ = run_experimental_analysis()
    
    for i, size in enumerate(data_sizes):
        quick_ratio = quick_results['times'][i] / builtin_results['times'][i]
        print(f"{size:8} {bubble_results['times'][i]:12.3f} {quick_results['times'][i]:12.3f} "
              f"{builtin_results['times'][i]:14.3f} {quick_ratio:15.2f}x")
    
    return builtin_results

In [ ]:
def run_part2_analysis():
    """Main function to run all Part 2 analysis"""
    
    print("🚀 PART 2: EXPERIMENTAL ALGORITHM ANALYSIS")
    print("=" * 70)
    
    # Step 1: Computational Complexity
    bubble_analysis, quick_analysis = generate_complexity_report()
    
    # Step 2: T(n) and Order of Growth
    sizes, t_bubble, t_quick = analyze_growth_rates()
    
    # Step 3 & 4: Experimental Analysis with Charts
    bubble_results, quick_results, data_sizes = run_experimental_analysis()
    create_growth_comparison_charts(bubble_results, quick_results, data_sizes)
    
    # Step 5: Compare with existing approach
    builtin_results = compare_with_builtin_sort(data_sizes)
    
    # Generate final summary report
    generate_final_summary(bubble_results, quick_results, builtin_results, data_sizes)
    
    print(f"\n✅ PART 2 ANALYSIS COMPLETED SUCCESSFULLY!")
    print(f"📊 Check generated charts:")
    print(f"   - growth_analysis_comprehensive.png")
    print(f"   - performance_comparison.png")

def generate_final_summary(bubble_results, quick_results, builtin_results, data_sizes):
    """Generate final summary report"""
    
    print(f"\n📋 FINAL SUMMARY - ALGORITHM ANALYSIS")
    print("=" * 60)
    
    # Performance at largest size
    max_size = data_sizes[-1]
    max_idx = data_sizes.index(max_size)
    
    print(f"\n🏆 Performance at n={max_size}:")
    print(f"   Bubble Sort: {bubble_results['times'][max_idx]:.3f} ms")
    print(f"   Quick Sort:  {quick_results['times'][max_idx]:.3f} ms")
    print(f"   Built-in:    {builtin_results['times'][max_idx]:.3f} ms")
    
    # Calculate improvements
    bubble_vs_quick = bubble_results['times'][max_idx] / quick_results['times'][max_idx]
    quick_vs_builtin = quick_results['times'][max_idx] / builtin_results['times'][max_idx]
    
    print(f"\n📈 Improvement Ratios:")
    print(f"   Quick Sort is {bubble_vs_quick:.1f}x faster than Bubble Sort")
    print(f"   Built-in is {1/quick_vs_builtin:.1f}x faster than Quick Sort")
    
    print(f"\n💾 Space Complexity:")
    print(f"   Bubble Sort: O(1) - Constant space")
    print(f"   Quick Sort:  O(log n) - Recursive stack")
    print(f"   Built-in:    O(n) - Requires additional space")

# Run the complete analysis
if __name__ == "__main__":
    run_part2_analysis()

In [ ]:
quickSort(Array, start, end):
    if start >= end:
        return
    pivot = Array[end]
    i = start - 1
    for j from start to end-1:
        if Array[j] <= pivot:
            i = i + 1
            swap(Array[i], Array[j])
    
    swap(Array[i + 1], Array[end])
    pivot_index = i + 1
    
    QUICKSORT(Array, start, pivot_index - 1)  
    QUICKSORT(Array, pivot_index + 1, end)
